In [ ]:
# Selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#Resolvendo problema do chromeDrive
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
# Pandas
import pandas as pd
# RegEx / Regular Expressions
import re
# Operating system
import os
import time

In [ ]:
case = 'https://loja.caseih.com.br/search?text='
nh = 'https://loja.newhollandag.com.br/search?text='
notFound = pd.DataFrame()
notFound["Produto"] = ''
notFound['Motivo'] = ''
cwd = os.getcwd()

In [ ]:
query = pd.read_csv('produtos_bot_yury_not_vv.txt', encoding = 'latin1', sep=',')
df_query = pd.DataFrame(query)
df_query.drop('IDX', axis=1, inplace=True)

In [ ]:
def contains_number(string):
    return any(char.isdigit() for char in string)

In [ ]:
def findOEM(text):
    print(text)
    oem = []
    if 'VV' in text:
        indice = text.find('VV')
        text = text.replace(text[indice:], '')
    if 'MCL' in text:
        cod_mcl = text.find('MCL')
        copy_text = text
        text = text[:cod_mcl]
        cod_mcl = copy_text[cod_mcl:]
        cod_mcl = cod_mcl.replace(' ', '')
        if contains_number(cod_mcl):
            if len(cod_mcl) > 5:
                oem.append(cod_mcl)
    if not contains_number(text):
        return oem
    num = re.search(r'\d+', text)
    indice = text.find(num.group())
    lastspace = text.rfind(' ', 0, indice)
    remover = [text[:lastspace], '.', '-']
    for rm in remover:
        text = text.replace(rm, '')
    text = text.replace(' ', '/')
    text = text.split('/')
    for txt in text:
        if contains_number(txt):
            if len(txt) > 5:
                if not contains_number(txt[-1]):
                    txt = txt.replace(txt[-1], '')
                oem.append(txt)
    return oem

In [ ]:
def nh_case(nav, cod, index):
    try:
        btn = nav.find_element(By.CLASS_NAME, 'product-card')
        link = btn.find_element(By.CLASS_NAME, 'stretched-link').get_attribute("href")
        nav.get(link)
        nome = nav.find_element(By.CLASS_NAME, 'product-title__product-name').text
#         nome = re.sub('[0-9]', '', nome)
        codigo = nav.find_element(By.CLASS_NAME, 'product-code').text
        elimg = nav.find_elements(By.CLASS_NAME, 'owl-carousel__images')
        src = []
        for img in elimg:
            img = img.get_attribute('src')
            if 'loja' not in img:
                src.append(img)
        print(len(src))
        src = list(dict.fromkeys(src))
        keys = []
        values = []
        for k in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__title'):
            keys.append(k.text)
        for v in nav.find_elements(By.CLASS_NAME, 'product-specifications-item__value'):
            values.append(v.text)
        dict_infos = dict(zip(keys, values))
        
        print(nome)
        print(codigo)
        print(len(src))
        for img in src:
            print(img)
        print(dict_infos)
    except:
        pass
def nhCase(nav, cod, index):
    try:
        nav.get(str(nh + cod))
        nh_case(nav, cod, index)       
        nav.get(str(case + cod))
        nh_case(nav, cod, index)
    except:
        pass

In [ ]:
chrome_options = webdriver.ChromeOptions()
servico = Service(ChromeDriverManager().install())
#chrome_options.add_argument('headless')
nav = webdriver.Chrome(options=chrome_options, service=servico)

In [ ]:
produtos = dict(zip(df_query['ET0100_PRODUTO'], df_query['ET0100_NOME']))
for ID, cod in produtos.items():
    cods = findOEM(cod)
    for item in cods:
        nhCase(nav, item, ID)

In [ ]:
nhCase(nav,'84257511', '1')

In [ ]:
display(notFound)

In [ ]:
notFound.to_excel(f'{cwd}/Commersul/notFound.xlsx')